In [155]:
import pandas as pd
import numpy as np

In [77]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [129]:
df = pd.read_csv('/content/final_data_kw - final_data_kw.csv', usecols= ['Description','Interests'])

In [130]:
df.Interests = df.Interests.str.rstrip("\r")
df.Interests = df.Interests.str.rstrip("\n")
df.Interests = df.Interests.str.rstrip("\r\n")

In [131]:
df.head()

,Description,Interests
0,Create a high-quality first person shooter gam...,Others
1,Machine learning is transforming the world aro...,Machine Learning
2,"KM3NeT, a new generation of neutrino telescope...",Science
3,A novel approach for the fusion of detection s...,Science
4,This book shows JavaScript developers how to b...,Web Dev


In [132]:
df = df[:250]

In [133]:
df.isnull().sum()

Description    0
Interests      0
dtype: int64

In [134]:
df_train, df_test = train_test_split(df, test_size=0.2)

In [135]:
df_train.Interests.nunique()

13

In [136]:
encoded_dict = {'Ethics':0,'Machine Learning':1,'Deep Learning':2,
                'Artificial Intelligence':3,'DSA':4,'Business':5,
                'Science':6,'Cryptography':7,'CS Fundamentals':8,
                'Web Dev':9,'App Dev':10,'Technology':11,'Others':12}

In [137]:
df_train['Interests'] = df_train.Interests.map(encoded_dict)
df_test['Interests'] = df_test.Interests.map(encoded_dict)

In [138]:
df_test.head()

,Description,Interests
132,Intercept problems and challenges typically fa...,8
168,"Real-world examples of cross-browser, mobile, ...",10
69,Enter the era of medical research using mobile...,11
207,"When, in 1984–86, Richard P. Feynman gave his ...",11
78,We construct compositional continuous approxim...,6


In [105]:
ohe = OneHotEncoder()

In [74]:
y_train = ohe.fit([df_train.Interests])
y_test = ohe.fit([df_test.Interests])

In [139]:
y_train = to_categorical(df_train.Interests)
y_test = to_categorical(df_test.Interests)

In [140]:
df_train.Interests.unique()

array([ 2, 11, 10,  6,  9,  8,  4,  1,  0,  3, 12,  5,  7])

In [141]:
df_test.Interests.unique()

array([ 8, 10, 11,  6,  1,  5,  9,  0,  2, 12,  4,  7,  3])

In [142]:
y_train

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [143]:
!pip install transformers
import transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [170]:
# from transformers import AutoTokenizer,TFBertModel

from transformers import DistilBertTokenizer, TFDistilBertModel
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
bert = TFDistilBertModel.from_pretrained("distilbert-base-uncased")

# tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
# bert = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [171]:
bert.summary()

Model: "tf_distil_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
Total params: 66,362,880
Trainable params: 66,362,880
Non-trainable params: 0
_________________________________________________________________


In [172]:
x_train = tokenizer(
    text = df_train.Description.tolist(),
    add_special_tokens = True,
    max_length = 100,
    truncation = True,
    padding = True,
    return_tensors = 'tf',
    return_attention_mask = True,
    verbose = True
)

x_test = tokenizer(
    text=df_test.Description.tolist(),
    add_special_tokens=True,
    max_length=100,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_attention_mask = True,
    verbose = True
)

In [173]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [174]:
x_train['input_ids']

<tf.Tensor: shape=(200, 100), dtype=int32, numpy=
array([[  101, 16934,  8285, ...,  4081,  2478,   102],
       [  101,  4553,  1996, ...,     0,     0,     0],
       [  101,  3443,  1996, ...,     0,     0,     0],
       ...,
       [  101,  2096, 11416, ..., 20611,  1999,   102],
       [  101,  4553,  9262, ...,     0,     0,     0],
       [  101,  2057,  2556, ...,  3550,  3614,   102]], dtype=int32)>

In [175]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [176]:
max_len = 100
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(64, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(16,activation = 'relu')(out)
y = Dense(13,activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [177]:
optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [180]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
  epochs=5,
    batch_size=8
)

Epoch 1/5
25/25 [==============================] - 3s 114ms/step - loss: 0.6055 - balanced_accuracy: 0.8400 - val_loss: 2.4553 - val_balanced_accuracy: 0.2800
Epoch 2/5
25/25 [==============================] - 3s 112ms/step - loss: 0.5507 - balanced_accuracy: 0.8450 - val_loss: 2.4364 - val_balanced_accuracy: 0.3400
Epoch 3/5
25/25 [==============================] - 3s 112ms/step - loss: 0.5003 - balanced_accuracy: 0.9100 - val_loss: 2.4707 - val_balanced_accuracy: 0.3000
Epoch 4/5
25/25 [==============================] - 3s 113ms/step - loss: 0.4815 - balanced_accuracy: 0.8900 - val_loss: 2.5215 - val_balanced_accuracy: 0.3000
Epoch 5/5
25/25 [==============================] - 3s 112ms/step - loss: 0.4199 - balanced_accuracy: 0.9250 - val_loss: 2.6204 - val_balanced_accuracy: 0.2600


In [181]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})

In [182]:
y_predicted = np.argmax(predicted_raw, axis = 1)
y_true = df_test.Interests

In [183]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_predicted))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.10      0.40      0.16         5
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         2
           5       0.40      1.00      0.57         2
           6       0.67      0.67      0.67         6
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         7
           9       1.00      0.43      0.60         7
          10       0.50      0.25      0.33         4
          11       0.17      0.14      0.15         7
          12       0.00      0.00      0.00         1

    accuracy                           0.26        50
   macro avg       0.22      0.22      0.19        50
weighted avg       0.31      0.26      0.25        50



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [190]:
df['Description'][75]

"In this course, you will dive into the concepts of rationality and irrationality and understand how they impact our investment decisions and what the consequences can be at the market level. You will first explore the different biases that we, as humans, are subjected to when facing investment decisions and how they may impact the outcomes of these decisions. Moreover, you will see how emotions and ethical concerns such as honesty and trust influence market participants. When they are considered as a group rather than individually, you will discover how rationality and irrationality can drive asset prices to and away from their fair value. Finally, you will be presented with different portfolio construction methodologies and investment styles that make up the landscape of today's portfolio management industry. At key points throughout the course, you will benefit from the practical knowledge of experts from our corporate partner, UBS, in how to build and manage clients' portfolios."

In [187]:
texts = """
Emphatic Temporal Difference (TD) methods are a class of off-policy
Reinforcement Learning (RL) methods involving the use of followon traces.
Despite the theoretical success of emphatic TD methods in addressing the
notorious deadly triad (Sutton and Barto, 2018) of off-policy RL, there are
still three open problems. First, the motivation for emphatic TD methods
proposed by Sutton et al. (2016) does not align with the convergence analysis
of Yu (2015). Namely, a quantity used by Sutton et al. (2016) that is expected
to be essential for the convergence of emphatic TD methods is not used in the
actual convergence analysis of Yu (2015). Second, followon traces typically
suffer from large variance, making them hard to use in practice. Third, despite
the seminal work of Yu (2015) confirming the asymptotic convergence of some
emphatic TD methods for prediction problems, there is still no finite sample
analysis for any emphatic TD method for prediction, much less control. In this
paper, we address those three open problems simultaneously via using truncated
followon traces in emphatic TD methods. Unlike the original followon traces,
which depend on all previous history, truncated followon traces depend on only
finite history, reducing variance and enabling the finite sample analysis of
our proposed emphatic TD methods for both prediction and control.
"""
x_val = tokenizer(
    text=texts,
    add_special_tokens=True,
    max_length=100,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True) 
validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
for key , value in zip(encoded_dict.keys(),validation[0]):
    print(key,value)

Ethics 59.284637
Machine Learning 96.42202
Deep Learning 3.1621125
Artificial Intelligence 99.76628
DSA 93.42939
Business 11.811188
Science 48.860676
Cryptography 0.09384114
CS Fundamentals 0.21196766
Web Dev 49.80603
App Dev 1.863425
Technology 91.03357
Others 95.51445


In [188]:
model.save('my_model.h5')

In [189]:
new_model = tf.keras.models.load_model('/content/my_model.h5', custom_objects={'TFDistilBertModel': TFDistilBertModel})

# Show the model architecture
new_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 100)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 tf_distil_bert_model_1 (TFDist  TFBaseModelOutput(l  66362880   ['input_ids[0][0]',              
 ilBertModel)                   ast_hidden_state=(N               'attention_mask[0][0]']         
                                one, 100, 768),                                                   
                                 hidden_states=None                                         

In [191]:
texts = """
In this course, you will dive into the concepts of rationality and irrationality and understand how they impact our investment decisions and what the consequences can be at the market level. You will first explore the different biases that we, as humans, are subjected to when facing investment decisions and how they may impact the outcomes of these decisions. Moreover, you will see how emotions and ethical concerns such as honesty and trust influence market participants. When they are considered as a group rather than individually, you will discover how rationality and irrationality can drive asset prices to and away from their fair value. Finally, you will be presented with different portfolio construction methodologies and investment styles that make up the landscape of today's portfolio management industry. At key points throughout the course, you will benefit from the practical knowledge of experts from our corporate partner, UBS, in how to build and manage clients' portfolios.
"""
x_val = tokenizer(
    text=texts,
    add_special_tokens=True,
    max_length=100,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True) 
validation = new_model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
for key , value in zip(encoded_dict.keys(),validation[0]):
    print(key,value)

Ethics 94.79464
Machine Learning 64.96689
Deep Learning 51.561813
Artificial Intelligence 22.39019
DSA 72.416245
Business 99.73463
Science 35.465553
Cryptography 1.6020464
CS Fundamentals 19.55917
Web Dev 37.53908
App Dev 71.733025
Technology 0.07447667
Others 92.978836
